## 11.1 교차검증 모델 만들기
실전에서 모델이 얼마나 잘 작동할지 평가하고 싶습니다.

데이터 전처리 파이프라인을 만들고 모델을 훈련한 다음 교차검증으로 평가합니다.

In [4]:
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# 숫자 데이터셋을 로드합니다.
digits = datasets.load_digits()

# 특성 행렬을 만듭니다.
features = digits.data

# 타깃 벡터를 만듭니다.
target = digits.target

# 표준화 객체를 만듭니다.
standardizer = StandardScaler()

# 로지스틱 회귀 객체를 만듭니다.
logit = LogisticRegression()

# 표준화한 다음 로지스틱 회귀를 실행하는 파이프라인을 만듭니다.
pipeline = make_pipeline(standardizer, logit)

# k-폴드 교차검증을 만듭니다.
kf = KFold(n_splits=10, shuffle=True, random_state=1)

# k-폴드 교차검증을 수행합니다.
cv_results = cross_val_score(pipeline, # 파이프라인
                             features, # 특성행렬
                             target, # 타깃 벡터
                             cv=kf, # 교차검증 기법
                             scoring="accuracy", # 평가 지표
                             n_jobs=-1) # 모든 CPU 코어 사용

# 평균을 계산합니다.
cv_results.mean()

0.9693916821849783

k-폴드 교차검증(k-fold cross-validation, KFCV)에서는 데이터를 fold라고 부르는 $k$개의 부분으로 나눕니다.$k$-1개 폴드를 하나의 훈련 세트로 합쳐 모델을 훈련하고 남은 폴드를 테스트 세트처럼 사용합니다. 이를 $k$번 반복합니다. 반복마다 다른 폴드를 테스트 세트로 사용합니다. $k$번 반복에서 얻은 모델 성능을 평균하여 최종 성능을 산출합니다.

해결에서 10개의 폴드를 사용하여 $k$-폴드 교차검증을 수행했습니다. 평가 정수는 cv_results에 저장되어 있습니다.

In [5]:
# 10개 폴드의 점수를 모두 확인하기
cv_results

array([0.97777778, 0.98888889, 0.96111111, 0.94444444, 0.97777778,
       0.98333333, 0.95555556, 0.98882682, 0.97765363, 0.93854749])

KFCV를 사용할 때 고려해야 할 중요한 점이 세 가지 있습니다. 첫째, KFCV는 각 샘플이 다른 샘플과 독립적으로 생성되었다고 가정합니다(즉 데이터는 independent identically distributed, iid입니다). 데이터가 iid라면 폴드를 나누기 전에 샘플을 섞는 것이 좋은 생각입니다. 사이킷런에서는 shuffle=True로 지정하여 섞을 수 있습니다.

둘째, KFCV를 사용하여 classifier를 평가할 때, 각 타깃 클래스의 샘플이 거의 같은 비율로 폴드에 담기는 것이 좋습니다(계층별 $k$-폴드, stratified k-fold라고 부릅니다). 예를 들어 성별 타깃 벡터 중에서 80% 샘플이 남성이라면 각 폴드도 80% 남성과 20% 여성 샘플로 이루어져야 합니다. 사이킷런에서는 KFold 클래스를 StratifiedKFold로 바꾸어 계층별 $k$-폴드 교차검증을 수행할 수 있습니다.

마지막으로 검증 세트나 교차검증을 사용할 때 훈련 세트에서 데이터를 전처리하고 이 변환을 훈련 세트와 테스트 세트에 모두 적용하는 것이 중요합니다. 예를 들면 표준화 객체 standardizer의 fit 메서드를 호출하여 훈련 세트의 평균과 분산을 계산합니다. 그다음 이 변환을 (transform 메서드를 사용해) 훈련 세트와 테스트 세트에 모두 적용합니다.

In [7]:
from sklearn.model_selection import train_test_split

# 훈련 세트와 테스트 세트를 만듭니다.
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.1, random_state=1)

# 훈련 세트로 standardizer의 fit 메서드를 호출합니다.
standardizer.fit(features_train)

# 훈련 세트와 테스트 세트에 모두 적용합니다.
features_train_std = standardizer.transform(features_train)
features_test_std = standardizer.transform(features_test)

이렇게 하는 이유는 테스트 세트를 모르는 척하기 위해서입니다. 이 전처리 객체를 훈련 세트와 테스트 세트에 있는 모든 샘플로 훈련한다면 테스트 세트의 정보가 훈련 세트로 유출된 것입니다. 이 규칙은 특성 선택 같은 모든 전처리 단계에 적용됩니다.

사이킷런의 pipeline 패키지는 교차검증 기법을 사용할 때 이 규칙을 손쉽게 구현할 수 있도록 도와줍니다. 먼저 데이터를 전처리 (예를 들면 standardizer)하고 모델(로지스틱 회귀인 logit)을 훈련하는 파이프라인을 만듭니다.

In [8]:
# 파이프라인을 만듭니다.
pipeline = make_pipeline(standardizer, logit)

그다음 이 파이프라인으로 KFCV를 실행하면 사이킷런이 모든 작업을 알아서 처리합니다.

In [9]:
# k-폴드 교차검증 수행
cv_results = cross_val_score(pipeline, # 파이프라인
                             features, # 특성행렬
                             target, # 타깃 벡터
                             cv=kf, # 교차검증 기법
                             scoring="accuracy", # 평가 지표
                             n_jobs=-1) # 모든 CPU 코어 사용

cross_val_score에는 아직 이야기하지 않은 중요한 세 개의 매개변수가 있습니다. cv는 교차검증 기법을 결정합니다. $k$-폴드를 가장 많이 사용하지만 다른 방식도 있습니다. LOOCV(leave-one-out-cross-validation)는 폴드의 수 $k$가 샘플의 개수와 같습니다. scoring 매개변수는 이 장의 다른 여러 레시피에서 설명할 모델 성공의 측정 방법을 결정합니다. 마지막으로 n_jobs=-1은 사이킷런에게 가용한 모든 코어를 사용하도록 지시합니다. 예를 들어 사용하는 컴퓨터에 (요즘 노트북의 기본 사양인) 네 개의 코어가 있다면 사이킷런은 네 개의 코어를 모두 동시에 사용해 작업의 속도를 높입니다.

LOOCV는 LeaveOneOut 클래스에 구현되어 있습니다. LeaveOneOut 클래스는 KFold(n_splits=n)과 동일합니다(n은 샘플 개수).

KFold와 StratifiedKFold의 n_splits 매개변수 기본값은 5입니다.

ShuffleSplit는 반복 횟수에 상관없이 훈련 폴드와 테스트 폴드 크기를 임의로 지정할 수 있습니다. train_size, test_size 매개변수에는 사용할 샘플 개수 또는 비율을 입력합니다. 반복마다 랜덤하게 분할하기 때문에 하나의 샘플이 여러 번 테스트 폴드에 포함될 수 있습니다. 계층별 교차검증을 위한 StratifiedShuffleSplit도 있습니다. 다음 코드는 훈련 폴드로 50%, 테스트 폴드로 20%를 사용하여 10번 반복하는 예입니다.

In [10]:
from sklearn.model_selection import ShuffleSplit

# ShuffleSplit 분할기를 만듭니다.
ss = ShuffleSplit(n_splits=10, train_size=0.5, test_size=0.2, random_state=42)

# 교차검증을 수행합니다.
cv_results = cross_val_score(pipeline,
                             features,
                             target,
                             cv=ss,
                             scoring="accuracy",
                             n_jobs=-1)

# 평균을 계산합니다.
cv_results.mean()

0.9630555555555554

사이킷런 0.19 버전에서는 교차검증을 반복하여 실행할 수 있는 RepeatedKFold와 StratifiedRepeatedKFold가 추가되었습니다. 다음 코드는 10-폴드 교차검증을 5번 반복하는 RepeatedKFold의 예입니다.

In [11]:
from sklearn.model_selection import RepeatedKFold

# RepeatedKFold 분할기를 만듭니다.
rfk = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)

# 교차검증을 수행합니다.
cv_results = cross_val_score(pipeline,
                             features,
                             target,
                             cv=rfk,
                             scoring="accuracy",
                             n_jobs=-1)

# 검증 점수 개수를 확인합니다.
len(cv_results)

50

총 50개의 교차검증 점수가 생성되었습니다. n_splits 매개변수 기본값은 5이고 n_repeats 매개변수 기본값은 10입니다.

## 11.2 기본 회귀 모델 만들기
다른 모델과 비교하기 위해 간단한 기본 회귀 모델을 만들고 싶습니다.

사이킷런의 DummyRegressor를 사용하여 기본 모델로 사용할 간단한 dummy 모델을 만듭니다.

In [6]:
# from sklearn.datasets import load_boston
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

data_url = "http://lib.stat.cmu.edu/datasets/boston"
boston = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
features = np.hstack([boston.values[::2, :], boston.values[1::2, :2]])
target = boston.values[1::2, 2]

# 데이터를 로드합니다.
# boston = load_boston()

# 특성을 만듭니다.
# features, target = boston.data, boston.target

# 훈련 세트와 테스트 세트를 나눕니다.
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=0)

# 더미 회귀 모델을 만듭니다.
dummy = DummyRegressor(strategy='mean')

# 더미 회귀 모델을 훈련합니다.
dummy.fit(features_train, target_train)

# R^2 점수를 계산합니다.
dummy.score(features_test, target_test)

-0.001119359203955339

다른 모델을 훈련하고 평가하여 성능 점수를 비교합니다.

In [8]:
from sklearn.linear_model import LinearRegression

# 간단한 선형 회귀 모델을 훈련합니다.
ols = LinearRegression()
ols.fit(features_train, target_train)

# R^2 점수를 계산합니다.
ols.score(features_test, target_test)

0.635463843320215

DummyRegressor 클래스는 실제 모델과 비교하기 위해 사용할 수 있는 매우 간단한 모델을 만듭니다. 기존 제품이나 시스템의 단순한 예측 방식을 흉내 내는 데 종종 사용합니다. 예를 들면 모든 새로운 사용자는 특성에 상관없이 첫 달에 100달러를 사용할 것이라고 가정하도록 애초부터 시스템에 하드코딩되어 있을지 모릅니다. 이런 가정을 기본 모델로 삼으면 머신러닝 방식을 사용했을 때 장점을 명확히 확인할 수 있습니다.

DummyRegressor 클래스는 strategy 매개변수를 사용하여 예측 방법을 지정합니다. 훈련 세트의 평균 또는 중간값을 사용할 수 있습니다. 또한 strategy를 constant로 지정하고 constant 매개변수를 사용하면 모든 샘플에 대해 일정한 값으로 예측하는 더미 회귀 모델을 만들 수 있습니다.

In [9]:
# 모든 샘플에 대해 20으로 예측하는 더미 회귀 모델을 만듭니다.
clf = DummyRegressor(strategy='constant', constant=20)
clf.fit(features_train, target_train)

# 점수를 계산합니다.
clf.score(features_test, target_test)

-0.06510502029325727

score 메서드에 대해 언급할 것이 하나 있습니다. 기본적으로 score 메서드는 결정계수($R^2$)값을 반환합니다. $$R^2 = 1-\frac{\sum_{i}(y_i -\hat{y_i})^2}{\sum_i(y_i-\bar{y})^2}$$
여기에서 $y_i$는 샘플의 정답 타깃값입니다. $\hat{y_i}$은 예측한 값이고 $\bar{y}$은 타깃 벡터의 평균값입니다. $R^2$이 1에 가까울수록 특성이 타깃 벡터의 분산을 잘 설명합니다.

strategy가 mean일 때 평균값으로 예측하고 median일 때 중간값으로 예측합니다. strategy='quantile'로 지정하면 quantile 매개변수에 지정한 분위값을 예측으로 사용합니다. quantile 매개변수에는 0과 1 사이의 실숫값을 지정하며 0.5일 때 중간값과 같고 0이면 최솟값, 1이면 최댓값입니다. 다음 코드는 훈련 세트의 타깃값의 최댓값으로 예측을 만드는 DummyRegressor의 예입니다.

In [11]:
clf = DummyRegressor(strategy='quantile', quantile=1.0)
clf.fit(features_train, target_train)

# 훈련 세트 타깃의 최댓값으로 예측합니다.
clf.predict(features_test)

array([50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50.,
       50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50.,
       50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50.,
       50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50.,
       50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50.,
       50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50.,
       50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50.,
       50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50.,
       50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50.,
       50., 50., 50., 50., 50., 50., 50., 50., 50., 50.])

In [12]:
import numpy as np
# 훈련 세트의 타깃에서 최댓값을 확인합니다.
np.max(target_train)

50.0

## 11.3 기본 분류 모델 만들기
다른 모델과 비교하기 위해 간단한 기본 회귀 모델을 만들고 싶습니다.

사이킷런의 DummyClassifier를 사용합니다.

In [16]:
from sklearn.datasets import load_iris
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split

iris = load_iris()

features, target = iris.data, iris.target

features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=0)

dummy = DummyClassifier(strategy='uniform', random_state=1)

dummy.fit(features_train, target_train)

dummy.score(features_test, target_test)

0.42105263157894735

훈련된 다른 모델과 기본 모델을 비교하여 더 나은지 확인할 수 있습니다.

In [17]:
from sklearn.ensemble import RandomForestClassifier

# 분류 모델을 만듭니다.
classifier = RandomForestClassifier()

# 모델을 훈련합니다.
classifier.fit(features_train, target_train)

# 정확도 점수를 계산합니다.
classifier.score(features_test, target_test)

0.9736842105263158

분류 모델의 성능을 측정하는 일반적인 방법은 랜덤한 추측보다 얼마나 더 나은지 비교하는 것입니다. 사이킷런의 DummyClassifier를 사용하면 이런 비교를 쉽게할 수 있습니다. strategy 매개변수는 예측값을 생성하는 여러 가지 옵션을 제공합니다. 전형적으로 많이 사용하는 두 가지 전략이 있습니다. 첫 번째, stratified 옵션은 훈련 세트에 있는 타깃 벡터의 클래스 비율에 비례하는 예측을 만듭니다(즉 훈련 세트에서 샘플의 20%가 여성이라면 DummyClassifier는 20%를 여성으로 예측합니다). 두 번째, uniform 옵션은 클래스 비중이 균등하도록 랜덤하게 예측합니다. 예를 들어 샘플의 20%가 여성이고 80%가 남성일 때 uniform 옵션은 50%는 여성, 50%는 남성으로 예측합니다.

strategy에서 자주 사용하는 또 다른 옵션은 most_frequent입니다. 이 옵션은 무조건 훈련 세트에서 가장 많은 타깃 레이블로 예측을 만듭니다. 다음 코드는 most_frequent 옵션을 사용하는 예입니다.

In [18]:
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(features_train, target_train)

# 훈련 세트 타깃에서 가장 많은 값으로 예측합니다.
dummy.predict(features_test)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

훈련 세트에 있는 타깃값을 확인해보면 클래스 레이블 2가 가장 많습니다.

In [19]:
# 훈련 세트의 타깃 개수를 확인합니다.
np.bincount(target_train)

array([37, 34, 41], dtype=int64)

## 11.4 이진 분류기의 예측 평가하기
훈련된 분류 모델의 품질을 평가하고 싶습니다.

사이킷런의 cross_val_score 함수를 사용하여 교차검증을 수행할 때 scoring 매개변수에 성능 지표 중 하나를 선택할 수 있습니다. 예를 들어, 정확도, 정밀도, 재현율, $F_1$이 있습니다.

정확도는 널리 사용되는 성능 지표입니다. 단순히 올바르게 예측된 샘플의 비율입니다.
$$정확도 = \frac{TP+TN}{TP+TN+FP+FN}$$

scoring="accuracy"로 지정하여 (기본값인) 5-폴드 교차검증의 정확도를 측정할 수 있습니다.

In [3]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification

X, y = make_classification(n_samples = 10000,
                           n_features = 3,
                           n_informative = 3,
                           n_redundant = 0,
                           n_classes = 2,
                           random_state = 1)

# 로지스틱 회귀 모델을 만듭니다.
logit = LogisticRegression()

# 정확도를 사용하여 교차검증을 수행합니다.
cross_val_score(logit, X, y, scoring='accuracy')

array([0.9555, 0.95  , 0.9585, 0.9555, 0.956 ])

정확도는 직관적이고 쉽게 설명할 수 있다는 것이 장점입니다. 정확도는 단순히 정확히 예측한 샘플의 비율입니다. 그러나 실전에서는 클래스 비율이 불균형한 데이터가 많습니다. 클래스가 불균형하면 모델의 정확도는 높지만 예측 성능이 나쁜 역설적인 상황이 발생합니다. 

정밀도는 양성으로 예측한 샘플 중에서 진짜 양성 클래스의 비율입니다. 이를 예측에 포함된 잡음이라고 생각할 수 있습니다. 즉 어떤 것을 양성 클래스로 예측했을 때 얼마나 올바른지를 나타냅니다. 높은 정밀도의 모델은 양성 클래스라고 확신이 높을 때만 양성 샘플로 예측합니다. 정밀도 공식은 다음과 같습니다.
$$precision = \frac{TP}{TP+FP}$$

In [4]:
# 정밀도를 사용한 교차검증
cross_val_score(logit, X, y, scoring='precision')

array([0.95963673, 0.94820717, 0.9635996 , 0.96149949, 0.96060606])

재현율은 진짜 양성 샘플 중에서 양성으로 예측한 비율입니다. 재현율은 모델이 양성 클래스 샘플을 구분하는 능력을 측정합니다. 높은 재현율의 모델은 샘플을 양성 클래스로 예측하기 위해서 낮은 기준을 가집니다.
$$recall = \frac{TP}{TP+FN}$$

In [5]:
# 재현율을 사용한 교차검증
cross_val_score(logit, X, y, scoring='recall')

array([0.951, 0.952, 0.953, 0.949, 0.951])

거의 항상 정밀도와 재현율 간의 균형을 맞추어야 합니다. 이를 위해 $F_1$ 점수가 만들어졌습니다. $F_1$은 정밀도와 재현율의 조화 평균입니다.
$$F_1 = 2 \times \frac{precision \times recall}{precision + recall}$$
이는 진짜 양성 레이블을 가진 샘플을 양성으로 성공적으로 예측한 정도를 측정합니다.

In [6]:
# f1 점수를 사용한 교차검증
cross_val_score(logit, X, y, scoring='f1')

array([0.95529884, 0.9500998 , 0.95827049, 0.95520886, 0.95577889])

평가 지표로서 정확도는 유용한 성질을 가지고 있습니다. 특히 이해하기 쉽습니다. 하지만 종종 정밀도와 재현율의 균형을 맞추는 것이 더 좋은 지표가 됩니다. 즉 비관적인 모델과 긍정적인 모델 사이의 트레이드오프입니다. $F_1$은 정밀도와 재현율을 비교적 동등하게 취급하여 이 둘 사이의 균형을 표현합니다.

cross_val_score를 사용하는 대신 진짜 y값과 예측한 y값이 있으면 직접 정확도와 재현율을 계산할 수 있습니다.

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 훈련 세트와 테스트 세트로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

# 테스트 세트의 예측을 만듭니다.
y_hat = logit.fit(X_train, y_train).predict(X_test)

# 정확도를 계산합니다.
accuracy_score(y_test, y_hat)

0.947

cross_val_score 함수의 cv 매개변수를 지정하지 않으면 회귀일 때는 KFold, 분류일 때는 StratifiedKFold 분할기가 사용됩니다. cv 매개변수에 정수를 입력하여 기본 분할기의 폴드 수를 지정할 수도 있습니다.

사이킷런 0.19 버전에서 cross_validate 함수가 추가되었습니다. 이 함수는 cross_val_score와 사용법이 매우 비슷하지만 scoring 매개변수에 여러 개의 평가 지표를 추가할 수 있습니다.

In [9]:
from sklearn.model_selection import cross_validate

# 정확도와 정밀도를 사용한 교차검증
cross_validate(logit, X, y, scoring=["accuracy", "precision"])

{'fit_time': array([0.03770471, 0.02892303, 0.02692604, 0.02495122, 0.01692486]),
 'score_time': array([0.014961  , 0.01097012, 0.01097035, 0.00599527, 0.00817347]),
 'test_accuracy': array([0.9555, 0.95  , 0.9585, 0.9555, 0.956 ]),
 'test_precision': array([0.95963673, 0.94820717, 0.9635996 , 0.96149949, 0.96060606])}

accuracy와 precision 두 개의 평가 지표에 대해 각각 다섯 개의 교차검증 점수가 반환되었습니다. 